## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "default"
study_type = "study"
include_test_users = False
labels = {}
use_imperial = True
sensed_algo_prefix = "cleaned"
survey_info = {}

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
# Do not run this notebook if it has trip-labels as ENKETO nbclient will run up through this cell
if survey_info.get('trip-labels', None) == 'ENKETO':
    ipython = get_ipython()
    ipython._showtraceback = scaffolding.no_traceback_handler
    raise Exception("The plots in this notebook are not relevant for ENKETO trip-labels")

In [ ]:
# get metric vs imperial vars
label_units, short_label, label_units_lower, distance_col, weight_unit = scaffolding.get_units(use_imperial)

### Color Dictionary

In [ ]:
colors_mode, colors_replaced, colors_purpose, colors_sensed, colors_ble = scaffolding.mapping_color_labels(labels)
values_to_translations, value_to_translations_purpose, values_to_translations_replaced = scaffolding.translate_values_to_labels(labels)

## Collect Data From Database for Generic Metrics

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = await scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            labels,
                                                                            include_test_users=include_test_users,
                                                                            add_footprint=True)

## Collect Data from Database for Sensed Metrics

In [ ]:
expanded_ct_sensed, file_suffix_sensed, quality_text_sensed, debug_df_sensed = await scaffolding.load_viz_notebook_sensor_inference_data(year,
                                                                            month,
                                                                            program,
                                                                            labels,
                                                                            include_test_users,
                                                                            sensed_algo_prefix)

## Collect Data from Database for Inferred Metrics

In [ ]:
expanded_ct_inferred, file_suffix_inferred, quality_text_inferred, debug_df_inferred = await scaffolding.load_viz_notebook_inferred_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            labels,
                                                                            include_test_users=include_test_users,
                                                                            add_footprint=True)

In [ ]:
merged_debug_df = debug_df.combine_first(debug_df_sensed)

In [ ]:
quality_text, quality_text_sensed

In [ ]:
import re
labeled_match = re.match(r'Based on ([0-9]+) confirmed trips from ([0-9]+) (users|testers and participants)\nof ([0-9]+) total  trips from ([0-9]+) (users|testers and participants) (\(([0-9.]+|nan)%\))', quality_text)
# labeled_match
stacked_bar_quality_text_labeled = f"{labeled_match.group(1)} trips {labeled_match.group(7)}\n from {labeled_match.group(2)} {labeled_match.group(3)}"

sensed_match = re.match(r'Based on ([0-9]+) trips from ([0-9]+) (users|testers and participants)', quality_text_sensed)
stacked_bar_quality_text_sensed = f"{sensed_match.group(1)} trips (100%)\n from {sensed_match.group(2)} {sensed_match.group(3)}"

inferred_match = re.match(r'Based on ([0-9]+) confirmed trips from ([0-9]+) (users|testers and participants)\nof ([0-9]+) total  trips from ([0-9]+) (users|testers and participants) (\(([0-9.]+|nan)%\))', quality_text_inferred)
stacked_bar_quality_text_inferred = f"{inferred_match.group(1)} trips {inferred_match.group(7)}\n from {inferred_match.group(2)} {inferred_match.group(3)}"

stacked_bar_quality_text_labeled, stacked_bar_quality_text_sensed, stacked_bar_quality_text_inferred

## 100% Stacked Bar Charts

### Distribution of modes

In [ ]:
file_name = f'ntrips_total{file_suffix}'
plot_title_no_quality= "Number of trips for each mode"

try:
    fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(15,3*2), sharex=True)
    # We will have text results corresponding to the axes for simplicity and consistency
    text_results = [["Unmodified Alt Text", "Unmodified HTML"], ["Unmodified Alt Text", "Unmodified HTML"], ["Unmodified Alt Text", "Unmodified HTML"]]
    
    plot_and_text_stacked_bar_chart(expanded_ct, lambda df: (df.groupby("mode_confirm_w_other").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False)), 
                                    "Labeled by user\n"+stacked_bar_quality_text_labeled, ax[0], text_results[0], colors_mode, debug_df, values_to_translations)
    plot_and_text_stacked_bar_chart(expanded_ct_inferred, lambda df: (df.groupby("mode_confirm_w_other").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False)), 
                                    "Inferred from prior labels\n"+stacked_bar_quality_text_inferred, ax[1], text_results[1], colors_mode, debug_df_inferred, values_to_translations)
    plot_and_text_stacked_bar_chart(expanded_ct_sensed, lambda df: (df.groupby("primary_mode").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False)), 
                                    "Sensed by OpenPATH\n"+stacked_bar_quality_text_sensed, ax[2], text_results[2], colors_sensed, debug_df_sensed)
    set_title_and_save(fig, text_results, plot_title_no_quality, file_name)
except (AttributeError, KeyError, pd.errors.UndefinedVariableError) as e:
    plt.clf()
    generate_missing_plot(plot_title_no_quality, merged_debug_df, file_name)
    alt_text = store_alt_text_missing(merged_debug_df, file_name, plot_title_no_quality)        
    alt_html = store_alt_html_missing(merged_debug_df, file_name, plot_title_no_quality)
except Exception as e:
    # TODO: Future cleanup can pass in just the figure and have the function choose the last axis
    fig, ax = plt.subplots()
    plot_and_text_error(e, ax, file_name)

### Distribution of modes in commute trips

In [ ]:
plot_title_no_quality= "Number of commute trips for each mode"
file_name = f"ntrips_commute_mode_confirm{file_suffix}"

try:
    trip_purpose_query = "purpose_confirm == 'work'"
    expanded_ct_commute = expanded_ct.query(trip_purpose_query)
    commute_quality_text = scaffolding.get_quality_text(expanded_ct, expanded_ct_commute, "commute", include_test_users) if not expanded_ct.empty else ""
    expanded_ct_inferred_commute = expanded_ct_inferred.query(trip_purpose_query)
    commute_quality_text_inferred = scaffolding.get_quality_text(expanded_ct_inferred, expanded_ct_inferred_commute, "commute", include_test_users) if not expanded_ct_inferred.empty else ""
    plot_title = plot_title_no_quality

    commute_labeled_match = re.match(r'Based on ([0-9]+) confirmed commute trips from ([0-9]+) (users|testers and participants)\nof ([0-9]+) total confirmed trips from ([0-9]+) (users|testers and participants) (\(([0-9.]+|nan)%\))', commute_quality_text)
    stacked_bar_quality_text_commute_labeled = f"{commute_labeled_match.group(1)} trips {commute_labeled_match.group(7)}\n from {commute_labeled_match.group(2)} {commute_labeled_match.group(3)}"

    commute_inferred_match = re.match(r'Based on ([0-9]+) confirmed commute trips from ([0-9]+) (users|testers and participants)\nof ([0-9]+) total confirmed trips from ([0-9]+) (users|testers and participants) (\(([0-9.]+|nan)%\))', commute_quality_text_inferred)
    stacked_bar_quality_text_commute_inferred = f"{commute_inferred_match.group(1)} trips {commute_inferred_match.group(7)}\n from {commute_inferred_match.group(2)} {commute_inferred_match.group(3)}"

    # Plot entries
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15,2*2), sharex=True)    
    text_results = [["Unmodified Alt Text", "Unmodified HTML"], ["Unmodified Alt Text", "Unmodified HTML"]]
    plot_and_text_stacked_bar_chart(expanded_ct_commute, lambda df: df.groupby("mode_confirm_w_other").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False), 
                                    "Labeled by user\n"+stacked_bar_quality_text_commute_labeled, ax[0], text_results[0], colors_mode, debug_df, values_to_translations)
    plot_and_text_stacked_bar_chart(expanded_ct_inferred_commute, lambda df: df.groupby("mode_confirm_w_other").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False), 
                                    "Inferred from prior labels\n"+stacked_bar_quality_text_commute_inferred, ax[1], text_results[1], colors_mode, debug_df_inferred, values_to_translations)
    set_title_and_save(fig, text_results, plot_title, file_name)
except (AttributeError, KeyError, pd.errors.UndefinedVariableError) as e:
    plt.clf()
    generate_missing_plot(plot_title_no_quality, debug_df, file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    
    alt_html = store_alt_html_missing(debug_df, file_name, plot_title_no_quality)
except Exception as e:
    fig, ax = plt.subplots()
    plot_and_text_error(e, ax, file_name)

### Distribution of Trip_purpose attribute

In [ ]:
plot_title_no_quality="Number of trips for each purpose"
file_name= f"ntrips_purpose{file_suffix}"
try:
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15,2*2), sharex=True)
    text_results = [["Unmodified Alt Text", "Unmodified HTML"], ["Unmodified Alt Text", "Unmodified HTML"]]
    plot_and_text_stacked_bar_chart(expanded_ct, lambda df: df.groupby("purpose_confirm_w_other").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False), 
                                    "Labeled by user\n"+stacked_bar_quality_text_labeled, ax[0], text_results[0], colors_purpose, debug_df, value_to_translations_purpose)
    plot_and_text_stacked_bar_chart(expanded_ct_inferred, lambda df: df.groupby("purpose_confirm_w_other").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False), 
                                    "Inferred from prior labels\n"+stacked_bar_quality_text_inferred, ax[1], text_results[1], colors_purpose, debug_df_inferred)
    set_title_and_save(fig, text_results, plot_title_no_quality, file_name)
except (AttributeError, KeyError, pd.errors.UndefinedVariableError) as e:
    plt.clf()
    generate_missing_plot(plot_title_no_quality, debug_df, file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)
    alt_html = store_alt_html_missing(debug_df, file_name, plot_title_no_quality)
except Exception as e:
    fig, ax = plt.subplots()
    plot_and_text_error(e, ax, file_name)

### Mode choice for trips under 80% mark

In [ ]:
file_name = f'ntrips_under80{file_suffix}'

try:
    # Preprocess to find cutoff and filter below cutoff
    # For simplicity, and to aid in comparison, we have a single cutoff based on the total number of trips
    cutoff = expanded_ct_sensed.distance.quantile(0.8)
    if pd.isna(cutoff):
        cutoff = 0
    dist_threshold = expanded_ct_sensed[distance_col].quantile(0.8).round(1)
    dist_threshold = str(dist_threshold) 

    plot_title_no_quality="Number of trips per travel model under " + dist_threshold + " " + label_units_lower
    plot_title_no_quality=plot_title_no_quality+"\n["+dist_threshold + " " + label_units_lower+" represents 80th percentile of trip length]"

    ## We do an existence check for the labeled df because we want to display the sensed value even if we don't have the labeled value
    ## but we don't need to have an existence check for sensed because in that case we will have no data to display
    expanded_ct_u80 = expanded_ct.loc[(expanded_ct['distance'] <= cutoff)] if "mode_confirm_w_other" in expanded_ct.columns else None
    expanded_ct_inferred_u80 = expanded_ct_inferred.loc[(expanded_ct_inferred['distance'] <= cutoff)] if "mode_confirm_w_other" in expanded_ct_inferred.columns else None
    expanded_ct_sensed_u80 = expanded_ct_sensed.loc[(expanded_ct_sensed['distance'] <= cutoff)]

    sensed_u80_quality_text = f"{len(expanded_ct_sensed_u80)} trips ({round(len(expanded_ct_sensed_u80)/len(expanded_ct_sensed)*100)}% of all trips)\nfrom {scaffolding.unique_users(expanded_ct_sensed_u80)} {sensed_match.group(3)}"
    labeled_u80_quality_text = f"{len(expanded_ct_u80)} trips ({round(len(expanded_ct_u80)/len(expanded_ct)*100)}% of all labeled,\n{round(len(expanded_ct_u80)/len(expanded_ct_sensed)*100)}% of all trips)\nfrom {scaffolding.unique_users(expanded_ct_u80)} {sensed_match.group(3)}" if "mode_confirm_w_other" in expanded_ct.columns else "0 labeled trips"
    inferred_u80_quality_text = f"{len(expanded_ct_inferred_u80)} trips ({round(len(expanded_ct_inferred_u80)/len(expanded_ct_inferred)*100)}% of all inferred,\n{round(len(expanded_ct_inferred_u80)/len(expanded_ct_sensed)*100)}% of all trips)\nfrom {scaffolding.unique_users(expanded_ct_inferred_u80)} {sensed_match.group(3)}" if "mode_confirm_w_other" in expanded_ct_inferred.columns else "0 inferred trips"

    # Plot entries
    fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(15,3*2), sharex=True)
    text_results = [["Unmodified Alt Text", "Unmodified HTML"], ["Unmodified Alt Text", "Unmodified HTML"], ["Unmodified Alt Text", "Unmodified HTML"]]
    plot_and_text_stacked_bar_chart(expanded_ct_u80, lambda df: df.groupby("mode_confirm_w_other").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False), 
                                    "Labeled by user\n"+labeled_u80_quality_text, ax[0], text_results[0], colors_mode, debug_df, values_to_translations)
    plot_and_text_stacked_bar_chart(expanded_ct_inferred_u80, lambda df: df.groupby("mode_confirm_w_other").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False), 
                                    "Labeled and Inferred by OpenPATH\n"+inferred_u80_quality_text, ax[1], text_results[1], colors_mode, debug_df_inferred, values_to_translations)
    plot_and_text_stacked_bar_chart(expanded_ct_sensed_u80, lambda df: df.groupby("primary_mode").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False), 
                                    "Sensed by OpenPATH\n"+sensed_u80_quality_text, ax[2], text_results[2], colors_sensed, debug_df_sensed)
    set_title_and_save(fig, text_results, plot_title_no_quality, file_name)
except (AttributeError, KeyError, pd.errors.UndefinedVariableError) as e:
    # we can have an missing attribute error during the pre-procssing, in which case we should show the missing plot
    # here, our pre-processing only relies on sensed data, so we use the debug_df_sensed
    plt.clf()
    plot_title_default = "Number of trips below 80th percentile in each mode"
    generate_missing_plot(plot_title_default, merged_debug_df, file_name)
    alt_text = store_alt_text_missing(merged_debug_df, file_name, plot_title_default)
    alt_html = store_alt_html_missing(merged_debug_df, file_name, plot_title_no_quality)
except Exception as e:
    fig, ax = plt.subplots()
    plot_and_text_error(e, ax, file_name)

### Total Trip Length covered by each mode

In [ ]:
plot_title_no_quality= label_units + " for each mode"
file_name =f"total_trip_length{file_suffix}"

try:
    fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(15,3*2), sharex=True)
    
    text_results = [["Unmodified Alt Text", "Unmodified HTML"], ["Unmodified Alt Text", "Unmodified HTML"], ["Unmodified Alt Text", "Unmodified HTML"]]
    plot_and_text_stacked_bar_chart(expanded_ct, lambda df: df.groupby("mode_confirm_w_other").agg({distance_col: 'sum'}).sort_values(by=distance_col, ascending=False), 
                                    "Labeled by user\n"+stacked_bar_quality_text_labeled, ax[0], text_results[0], colors_mode, debug_df, values_to_translations)
    plot_and_text_stacked_bar_chart(expanded_ct_inferred, lambda df: df.groupby("mode_confirm_w_other").agg({distance_col: 'sum'}).sort_values(by=distance_col, ascending=False), 
                                    "Inferred from prior labels\n"+stacked_bar_quality_text_inferred, ax[1], text_results[1], colors_mode, debug_df_inferred, values_to_translations)
    plot_and_text_stacked_bar_chart(expanded_ct_sensed, lambda df: df.groupby("primary_mode").agg({distance_col: 'sum'}).sort_values(by=distance_col, ascending=False), 
                                    "Sensed by OpenPATH\n"+stacked_bar_quality_text_sensed, ax[2], text_results[2], colors_sensed, debug_df_sensed)
    set_title_and_save(fig, text_results, plot_title_no_quality, file_name)    
except (AttributeError, KeyError, pd.errors.UndefinedVariableError) as e:
    plt.clf()
    generate_missing_plot(plot_title_no_quality, merged_debug_df, file_name)
    alt_text = store_alt_text_missing(merged_debug_df, file_name, plot_title_no_quality)  
    alt_html = store_alt_html_missing(merged_debug_df, file_name, plot_title_no_quality)
except Exception as e:
    fig, ax = plt.subplots()
    plot_and_text_error(e, ax, file_name)

### Total Trip Length covered by each land transport mode

In [ ]:
plot_title_no_quality= "Total trip length (" + label_units_lower + ") covered by each mode on land"
file_name =f"total_trip_length_land{file_suffix}"

try:
    ## We do an existence check for the labeled df because we want to display the sensed value even if we don't have the labeled value
    ## but we don't need to have an existence check for sensed because in that case we will have no data to display
    labeled_land_trips_df = expanded_ct[expanded_ct['base_mode'] != "AIR"] if "base_mode" in expanded_ct.columns else None
    inferred_land_trips_df = expanded_ct_inferred[expanded_ct_inferred['base_mode'] != "AIR"] if "base_mode" in expanded_ct_inferred.columns else None
    sensed_land_trips_df = expanded_ct_sensed[expanded_ct_sensed['primary_mode'] != "AIR_OR_HSR"]
    
    sensed_land_quality_text = f"{len(sensed_land_trips_df)} trips ({round(len(sensed_land_trips_df)/len(expanded_ct_sensed)*100)}% of all trips)\nfrom {scaffolding.unique_users(sensed_land_trips_df)} {sensed_match.group(3)}"
    labeled_land_quality_text = f"{len(labeled_land_trips_df)} trips ({round(len(labeled_land_trips_df)/len(expanded_ct_sensed)*100)}% of all trips)\nfrom {scaffolding.unique_users(labeled_land_trips_df)} {sensed_match.group(3)}" if "Mode_confirm" in expanded_ct.columns else "0 labeled trips"
    inferred_land_quality_text = f"{len(inferred_land_trips_df)} trips ({round(len(inferred_land_trips_df)/len(expanded_ct_inferred)*100)}% of all inferred,\n{round(len(inferred_land_trips_df)/len(expanded_ct_sensed)*100)}% of all trips)\nfrom {scaffolding.unique_users(inferred_land_trips_df)} {sensed_match.group(3)}" if "Mode_confirm" in expanded_ct_inferred.columns else "0 inferred trips"
    
    fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(15,3*2), sharex=True)
    plot_and_text_stacked_bar_chart(labeled_land_trips_df, lambda df: df.groupby("mode_confirm_w_other").agg({distance_col: 'sum'}).sort_values(by=distance_col, ascending=False), 
                                    "Labeled by user\n"+labeled_land_quality_text,  ax[0], text_results[0], colors_mode, debug_df, values_to_translations)
    plot_and_text_stacked_bar_chart(inferred_land_trips_df, lambda df: df.groupby("mode_confirm_w_other").agg({distance_col: 'sum'}).sort_values(by=distance_col, ascending=False), 
                                    "Inferred from prior labels\n"+inferred_land_quality_text, ax[1], text_results[1], colors_mode, debug_df_inferred, values_to_translations)
    plot_and_text_stacked_bar_chart(sensed_land_trips_df, lambda df: df.groupby("primary_mode").agg({distance_col: 'sum'}).sort_values(by=distance_col, ascending=False), 
                                    "Sensed by OpenPATH\n"+sensed_land_quality_text, ax[2], text_results[2], colors_sensed, debug_df_sensed)
    set_title_and_save(fig, text_results, plot_title_no_quality, file_name)    
except (AttributeError, KeyError, pd.errors.UndefinedVariableError) as e:
    plt.clf()
    generate_missing_plot(plot_title_no_quality, merged_debug_df, file_name)
    alt_text = store_alt_text_missing(merged_debug_df, file_name, plot_title_no_quality)        
    alt_html = store_alt_html_missing(merged_debug_df, file_name, plot_title_no_quality)
except Exception as e:
    fig, ax = plt.subplots()
    plot_and_text_error(e, ax, file_name)

## Generic Metrics (Bar Charts)

### Average miles per transport mode selected (Mode_confirm)

In [ ]:
plot_title_no_quality="Average "+ label_units+" for each mode with > 3 entries"
file_name ='average_miles_mode_confirm%s' % file_suffix

try:
    dist = expanded_ct.groupby('Mode_confirm').agg({distance_col: ['sum', 'count' , 'mean']})
    dist.columns = ['Total ('+label_units_lower+')', 'Count', 'Average ('+label_units_lower+')']
    dist = dist.reset_index()
    dist =dist.sort_values(by=['Total ('+label_units_lower+')'], ascending=False)

    x='Mode_confirm'
    y='Average ('+label_units_lower+')'
    plot_title= plot_title_no_quality+"\n"+quality_text
    
    data = dist.drop((dist.query("Count < 3").index)).sort_values(by=['Average ('+label_units_lower+')'], ascending=False)

    barplot_mode(data,x,y,plot_title, expanded_ct['Mode_confirm'].dropna().unique().tolist(), file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Average ('+label_units_lower+')'].values, data['Mode_confirm']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    

### Number of trips by day¶

In [ ]:
plot_title_no_quality="Number of trips by day"
file_name ='ntrips_per_day%s' % file_suffix

try:
    fq_days = expanded_ct.groupby(['start_local_dt_day']).agg({'start_local_dt_day': ['sum', 'count']})
    fq_days = fq_days.reset_index()
    fq_days.columns = ['Day of the Month', 'Total', 'Number of Trips']

    data = fq_days
    x = 'Day of the Month'
    y = 'Number of Trips'
    
    plot_title= plot_title_no_quality+"\n"+quality_text

    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Day of the Month']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    

### Number of trips by day of week¶

In [ ]:
plot_title_no_quality="Number of trips by weekday"
file_name ='ntrips_per_weekday%s' % file_suffix
try:
    fq_weekdays = expanded_ct.groupby(['start_local_dt_weekday']).agg({'start_local_dt_weekday': ['sum', 'count']})
    fq_weekdays = fq_weekdays.reset_index()
    fq_weekdays.columns = ['Weekday', 'Total', 'Number of Trips']
    weekday_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    fq_weekdays["Weekday"] = fq_weekdays.Weekday.apply(lambda x: weekday_labels[x])

    data = fq_weekdays
    x = 'Weekday'
    y = 'Number of Trips'

    plot_title= plot_title_no_quality+"\n"+quality_text
    
    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Weekday']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    